# YOLO v3 Object Detection

Let's see how to use the state of the art in object detection! Please make sure to watch the video, there is no code along here, since we can't reasonably train the YOLOv3 network ourself, instead we will use a pre-established version.

CODE SOURCE: https://github.com/xiaochus/YOLOv3

REFERENCE (for original YOLOv3): 

        @article{YOLOv3,  
              title={YOLOv3: An Incremental Improvement},  
              author={J Redmon, A Farhadi },
              year={2018} 
--------
----------
## YOU MUST WATCH THE VIDEO LECTURE TO PROPERLY SET UP THE MODEL AND WEIGHTS. THIS NOTEBOOK WON'T WORK UNLESS YOU FOLLOW THE EXACT SET UP SHOWN IN THE VIDEO LECTURE.
-------
-------

In [1]:
import os
import time
import cv2
import numpy as np
from model.yolo_model import YOLO

Using TensorFlow backend.


In [2]:
def process_image(img):
    """Resize, reduce and expand image.

    # Argument:
        img: original image.

    # Returns
        image: ndarray(64, 64, 3), processed image.
    """
    image = cv2.resize(img, (416, 416),
                       interpolation=cv2.INTER_CUBIC)
    image = np.array(image, dtype='float32')
    image /= 255.
    image = np.expand_dims(image, axis=0)

    return image

In [3]:
def get_classes(file):
    """Get classes name.

    # Argument:
        file: classes name for database.

    # Returns
        class_names: List, classes name.

    """
    with open(file) as f:
        class_names = f.readlines()
    class_names = [c.strip() for c in class_names]

    return class_names

In [30]:
def draw(image, boxes, scores, classes, all_classes):
    """Draw the boxes on the image.

    # Argument:
        image: original image.
        boxes: ndarray, boxes of objects.
        classes: ndarray, classes of objects.
        scores: ndarray, scores of objects.
        all_classes: all classes name.
    """
    for box, score, cl in zip(boxes, scores, classes):
        x, y, w, h = box

        top = max(0, np.floor(x + 0.5).astype(int))
        left = max(0, np.floor(y + 0.5).astype(int))
        right = min(image.shape[1], np.floor(x + w + 0.5).astype(int))
        bottom = min(image.shape[0], np.floor(y + h + 0.5).astype(int))

        cv2.rectangle(image, (top, left), (right, bottom), (255, 0, 0), 2)
        cv2.putText(image, '{0} {1:.2f}'.format(all_classes[cl], score),
                    (top, left - 6),
                    cv2.FONT_HERSHEY_SIMPLEX,
                    5, (0, 0, 255), 1,
                    cv2.LINE_AA)

        print('class: {0}, score: {1:.2f}'.format(all_classes[cl], score))
        print('box coordinate x,y,w,h: {0}'.format(box))

    print()

In [24]:
def detect_image(image, yolo, all_classes):
    """Use yolo v3 to detect images.

    # Argument:
        image: original image.
        yolo: YOLO, yolo model.
        all_classes: all classes name.

    # Returns:
        image: processed image.
    """
    pimage = process_image(image)

    start = time.time()
    boxes, classes, scores = yolo.predict(pimage, image.shape)
    end = time.time()

    print('time: {0:.2f}s'.format(end - start))

    if boxes is not None:
        draw(image, boxes, scores, classes, all_classes)

    return image

In [25]:
def detect_video(video, yolo, all_classes):
    """Use yolo v3 to detect video.

    # Argument:
        video: video file.
        yolo: YOLO, yolo model.
        all_classes: all classes name.
    """
    video_path = os.path.join("videos", "test", video)
    camera = cv2.VideoCapture(video_path)
    cv2.namedWindow("detection", cv2.WINDOW_AUTOSIZE)

    # Prepare for saving the detected video
    sz = (int(camera.get(cv2.CAP_PROP_FRAME_WIDTH)),
        int(camera.get(cv2.CAP_PROP_FRAME_HEIGHT)))
    fourcc = cv2.VideoWriter_fourcc(*'mpeg')

    
    vout = cv2.VideoWriter()
    vout.open(os.path.join("videos", "res", video), fourcc, 20, sz, True)

    while True:
        res, frame = camera.read()

        if not res:
            break

        image = detect_image(frame, yolo, all_classes)
        cv2.imshow("detection", image)

        # Save the video frame by frame
        vout.write(image)

        if cv2.waitKey(110) & 0xff == 27:
                break

    vout.release()
    camera.release()
    

In [26]:
yolo = YOLO(0.6, 0.5)
file = 'data/coco_classes.txt'
all_classes = get_classes(file)

C:\Anaconda3\envs\aind-dl\lib\site-packages\keras\models.py:240: UserWarning: No training configuration found in save file: the model was *not* compiled. Compile it manually.
  warnings.warn('No training configuration found in save file: '


### Detecting Images

In [27]:
f = 'jingxiang-gao-489454-unsplash.jpg'
path = 'images/test/'+f
image = cv2.imread(path)
image = detect_image(image, yolo, all_classes)
cv2.imwrite('images/res/' + f, image)

time: 19.85s
class: person, score: 0.64
box coordinate x,y,w,h: [2523.8918066  1482.56186676  619.408831   1302.60197306]
class: bicycle, score: 0.84
box coordinate x,y,w,h: [2877.69502401 2008.98068333 1303.69913578  717.74808955]



True

In [28]:
f = 'person.jpg'
path = 'images/test/'+f
image = cv2.imread(path)
image = detect_image(image, yolo, all_classes)
cv2.imwrite('images/res/' + f, image)

time: 21.07s
class: person, score: 1.00
box coordinate x,y,w,h: [187.67944336  83.12439895  91.77330017 306.58677053]
class: horse, score: 1.00
box coordinate x,y,w,h: [396.46743774 137.31078506 215.66383362 208.48716593]
class: dog, score: 1.00
box coordinate x,y,w,h: [ 61.27861977 263.36980581 145.18102646  88.4291451 ]



True

In [31]:
for fl in os.listdir('images/test/'):
    path = 'images/test/'+ fl
    image = cv2.imread(path)
    image = detect_image(image, yolo, all_classes)
    cv2.imwrite('images/res/' + fl, image)

time: 14.56s
class: bird, score: 1.00
box coordinate x,y,w,h: [1030.50643921  644.42308044  635.75809479  790.15713501]
class: bird, score: 1.00
box coordinate x,y,w,h: [1296.29104614  133.5457077   666.40586853  714.87199402]
class: bird, score: 1.00
box coordinate x,y,w,h: [1762.10494995  843.90664673  567.20115662  656.97688293]
class: bird, score: 0.98
box coordinate x,y,w,h: [1701.0333252   -11.24155426  446.21421814  442.3324585 ]
class: bird, score: 0.97
box coordinate x,y,w,h: [629.94884491  11.91158295 427.23228455 537.90623474]
class: bird, score: 0.73
box coordinate x,y,w,h: [249.50109482  -6.16536713 342.17693329 435.61720276]

time: 19.96s
class: apple, score: 0.68
box coordinate x,y,w,h: [ 97.09201813  22.59966373 454.42085266 431.51621819]

time: 11.60s
time: 11.14s
class: person, score: 0.99
box coordinate x,y,w,h: [ -74.17858887 -271.81511307 2452.6557312  3536.87805176]
class: dog, score: 0.96
box coordinate x,y,w,h: [1660.95030212  763.51341248 2885.52420044 2611.837